In [1]:
#Information systems and decision support systems and Journal of strategic information systems have same journal UI so I am using this code for both of them.
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
#initalize the chrome webdriver
driver=webdriver.Chrome()
driver.get('https://www.sciencedirect.com/journal/the-journal-of-strategic-information-systems/vol/34/issue/2')


# Wait for the page to fully load
driver.implicitly_wait(20)
data=[]
def getVolumedata(url):
    try:
        if not url:
            return 0
        driver.get(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        articles=driver.find_elements(By.CSS_SELECTOR,'li.js-article-list-item dl.js-article')
        vol_issue=driver.find_element(By.CLASS_NAME,'js-vol-issue').text.strip()
        vol_iss_year=driver.find_element(By.CLASS_NAME,'js-issue-status').text.split('(')[-1].strip(') ')
        
        for article in articles:
            article_url_element=article.find_element(By.CSS_SELECTOR,'a.article-content-title')
            article_url=article_url_element.get_attribute('href')
            article_title=article.find_element(By.CLASS_NAME,'js-article-title').text.strip()
            
            data.append([article_title,article_url,vol_issue,vol_iss_year])
        prev_ele=driver.find_element(By.CSS_SELECTOR,'nav.issue-navigation div.navigation-pre a.text-m')
        prev_ele_url=prev_ele.get_attribute('href')
        with open('Journal_of_SIS.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Title", "URL","Volume Issue","Vol Issue Year"])  # Write the header row
            writer.writerows(data)  # Write the article title and URL data rows
        return getVolumedata(prev_ele_url)
    except Exception as e:
        print("Exception",e)
       
try:
    articles=driver.find_elements(By.CSS_SELECTOR,'li.js-article-list-item dl.js-article')
    vol_issue=driver.find_element(By.CLASS_NAME,'js-vol-issue').text.strip()
    vol_iss_year=driver.find_element(By.CLASS_NAME,'js-issue-status').text.split('(')[-1].strip(') ')
    for article in articles:
        article_url_element=article.find_element(By.CSS_SELECTOR,'a.article-content-title')
        article_url=article_url_element.get_attribute('href')
        article_title=article.find_element(By.CLASS_NAME,'js-article-title').text.strip()
        
        
        data.append([article_title,article_url,vol_issue,vol_iss_year])
    prev_ele=driver.find_element(By.CSS_SELECTOR,'nav.issue-navigation div.navigation-pre a.text-m')
    prev_ele_url=prev_ele.get_attribute('href')
    with open('Journal_of_SIS.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Title", "URL",'Volume Issue','Month Year'])  # Write the header row
        writer.writerows(data)  # Write the article title and URL data rows
    getVolumedata(prev_ele_url)
except Exception as e:
    print("Exception",e)

driver.quit()


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import csv
import time
from selenium.webdriver.chrome.service import Service


#initalize the chrome webdriver

driver = webdriver.Chrome()
# journals_data=pd.read_csv('Journal_of_SIS.csv')
journals_data=pd.read_csv('decision_support_systems.csv')
# journal_urls=journals_data['URL']
# driver.get('https://www.sciencedirect.com/science/article/pii/0167923685901915')
# # Wait for the page to fully load
# driver.implicitly_wait(20)
# author_group=driver.find_elements(By.CSS_SELECTOR,'div.author-group')
# if author_group:
#     print('author group')
# else:
#     print('no author group')
def getAuthorsData(authors,driver):
    authdata=[]
    for auth in authors:
        name,desc='',''
        try:
            auth.click()
            # Wait for the side panel to open
            time.sleep(2)  
            auth_data=driver.find_element(By.CSS_SELECTOR,'div.side-panel-content')
            auth_data_desc=auth_data.find_element(By.CSS_SELECTOR,'div.affiliation')
            auth_name=auth_data.find_element(By.CSS_SELECTOR,'div.author')
            name=auth_name.text.strip()
            desc=auth_data_desc.text.strip()
        except Exception as e:
            print("error at author data",e)
        try:
            auth_email=auth_data.find_element(By.CSS_SELECTOR,'div.e-address a.anchor')
            email=auth_email.text.strip()
        except Exception as e:
            email=None
        authdata
        authdata.append([name,email,desc])
    return authdata
        


with open('dss_articles.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL','Journal_Title','Article_Title','Volume_Issue','Month_Year','Abstract','Keywords','Author_name','Author_email','Author_Address'])
    


for index,row in journals_data[:2].iterrows():
    # service = Service(r"C:\Users\H664K297\Documents\Journals\Project2\chromedriver-win64\chromedriver.exe")
    driver = webdriver.Chrome()
    final_data=[]
    url=row['URL']
    article_date=row['Vol Issue Year']
    driver.get(url)
    # Wait for the page to fully load
    driver.implicitly_wait(10)
    # Wait for the page to load using WebDriverWait
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.Head'))
        )
        title=driver.find_element(By.CSS_SELECTOR,'h1.Head').text.strip()
        article_details=driver.find_element(By.CSS_SELECTOR,'div.Publication')
        article_journal=article_details.find_element(By.CSS_SELECTOR,'h2.publication-title').text.strip()
        article_vol=article_details.find_element(By.CSS_SELECTOR,'div.text-xs a.anchor').text.strip()
                
        # article_month_year=article_details.find_element(By.CSS_SELECTOR,'div.text-xs').text.split(',')[1]
        
    except Exception as e:
        print(f"Error is {e} in this {url}")
    try:
        abstract=driver.find_element(By.ID,'div.sp0050').text.strip()
    except Exception as e:
        abstract=None
        print("error at abstraction",e)
    try:
        keywords=driver.find_elements(By.CSS_SELECTOR,'div.keyword')
        keyword_list=[]
        for key in keywords:
            keyword=key.text.strip()
            keyword_list.append(keyword)
    except Exception as e:
        print('error at keywords',e)

        
    final_data=[url,article_journal,title,article_vol,article_date]
    final_data.append(abstract)
    final_data.append(keyword_list)
    try:
        author_group=driver.find_element(By.CSS_SELECTOR,'div.author-group')
        authors=author_group.find_elements(By.CSS_SELECTOR,'button.button-link')
        auth_data=getAuthorsData(authors,driver)
        for i in auth_data:
            with open('dss_articles.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(final_data + i)
                file.flush()
    except Exception as e:
            with open('dss_articles.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(final_data + ["N/A", "N/A", "N/A"])
                file.flush()
            print(f"Error processing author data on {url}: {e}")
    driver.quit()


driver.quit()

error at abstraction Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="div.sp0050"]"}
  (Session info: chrome=132.0.6834.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF75E1D02F5+28725]
	(No symbol) [0x00007FF75E132AE0]
	(No symbol) [0x00007FF75DFC510A]
	(No symbol) [0x00007FF75E0193D2]
	(No symbol) [0x00007FF75E0195FC]
	(No symbol) [0x00007FF75E063407]
	(No symbol) [0x00007FF75E03FFEF]
	(No symbol) [0x00007FF75E060181]
	(No symbol) [0x00007FF75E03FD53]
	(No symbol) [0x00007FF75E00A0E3]
	(No symbol) [0x00007FF75E00B471]
	GetHandleVerifier [0x00007FF75E4FF30D+3366989]
	GetHandleVerifier [0x00007FF75E5112F0+3440688]
	GetHandleVerifier [0x00007FF75E5078FD+3401277]
	GetHandleVerifier [0x00007FF75E29AAAB+858091]
	(No symbol) [0x00007FF75E13E74F]
	(No symbol) [0x00007FF75E13A304]
	(No symbol) [0x00007FF75E1